In [1]:
"""Create a random forest model using the elemental features."""

import mendeleev
import numpy as np
import pandas as pd
# csv_data_source = '/home/dts/Documents/hu/jraph_MPEU/other_models/elem_rf/test_aflow_data.csv'

# Top one is for HLRN cluster, bottom for laptop.
# home_dir = '/home/bepdansp/jraph_MPEU/other_models'
home_dir = '/home/dts/Documents/hu/jraph_MPEU/other_models'
csv_data_source = '/band_gap_data/classification_data.csv'


# Read the data into a dataframe:
aflow_df = pd.read_csv(home_dir+csv_data_source, index_col=0)

aflow_df.describe()

,spacegroup_relax,Egap,nspecies
count,7018.000000,7018.000000,7018.0
mean,141.332431,2.429877,2.0
std,74.853533,1.884421,0.0
min,1.000000,0.010100,2.0
25%,62.000000,0.822225,2.0
50%,164.000000,2.031350,2.0
75%,206.000000,3.769925,2.0
max,230.000000,14.269400,2.0


In [2]:
aflow_df

,compound,auid,aurl,spacegroup_relax,Pearson_symbol_relax,Egap,catalog,nspecies
0,Pt4Yb4,aflow:a7634f582aafb904,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Pt1Yb...,62,oP8,0.0101,ICSD,2
1,Pt4Yb4,aflow:34e019ec63ae11ca,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Pt1Yb...,62,oP8,0.0111,ICSD,2
2,O30V16,aflow:adbf767994c54edf,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TRI/O15V8...,2,aP46,0.0166,ICSD,2
3,Se2Ti1,aflow:49194c177fb39709,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/HEX/Se2Ti...,164,hP3,0.0233,ICSD,2
4,O4Pb2,aflow:649e930cf5f6349d,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TET/O2Pb1...,136,tP6,0.0235,ICSD,2
...,...,...,...,...,...,...,...,...
7013,C2F8,aflow:2062cafdb8206a6b,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCLC/C1F4...,15,mS20,8.9097,ICSD,2
7014,C2F8,aflow:4b92322fb2f540cb,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCLC/C1F4...,15,mS20,9.0627,ICSD,2
7015,C2F8,aflow:f55c7391a1a53170,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCLC/C1F4...,15,mS20,9.0662,ICSD,2
7016,C2F8,aflow:a50c60f1699d218d,aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/MCLC/C1F4...,15,mS20,9.0840,ICSD,2


In [3]:
atomic_data_csv = home_dir + '/elem_rf/really_tight_full_cut20_revpbe.csv'
atomic_data_df = pd.read_csv(atomic_data_csv, index_col=0)
# atomic_data_df.describe()


sub_features_list = [
    'EA_half', 'IP_half', 'HOMO', 'LUMO', 'rs', 'rp', 's index', 'p index']
atomic_data_df[sub_features_list].describe()

,EA_half,IP_half,HOMO,LUMO,rs,rp,s index,p index
count,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000
mean,-0.133439,8.005121,-4.412885,-2.309800,1.406116,1.647025,25.156863,28.127451
std,1.359551,3.559766,2.365497,1.747167,0.506212,0.779726,14.989597,17.167591
min,-3.640255,3.862009,-15.794577,-7.092348,0.296000,0.253600,1.000000,3.000000
25%,-0.729464,5.777463,-5.509590,-3.280970,1.011200,1.017175,15.000000,16.000000
50%,-0.146451,6.960471,-3.744841,-2.088304,1.424900,1.714200,24.000000,25.000000
75%,0.577931,8.862555,-2.787848,-1.328261,1.800150,2.285250,40.000000,41.000000
max,3.747625,26.826802,-1.397061,2.031140,2.571500,4.467100,51.000000,57.000000


In [4]:
atomic_data_df['atomic_number'] = atomic_data_df.index

In [5]:
atomic_data_df

,Element,EA_half,IP_half,EA_delta,IP_delta,HOMO,LUMO,rs,s index,rp,p index,rd,d index,rf,f index,atomic_number
Atomic number,,,,,,,,,,,,,,,,
1,H,1.055959,13.993807,-1.818744,-1.081840,-6.543722,-6.543722,0.5403,1,1.3810,3,1.3338,7,0.0000,0,1
2,He,3.004933,26.826802,3.080201,27.066837,-15.794577,1.376831,0.2960,1,1.0300,3,0.6615,10,0.0000,0,2
3,Li,0.318876,5.690229,0.203820,5.607457,-3.227621,-1.285962,1.6320,2,1.9472,3,9.9999,14,6.7218,7,3
4,Be,0.774939,9.511429,0.684319,9.562825,-5.614316,-1.984163,1.0849,2,1.2169,3,2.9114,7,0.0000,0,4
5,B,-0.422340,8.341061,-0.587733,8.441792,-3.981294,-3.981294,0.8028,2,0.8370,3,2.1319,10,0.0000,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Cf,1.602567,6.175161,-1.433579,4.009092,-6.175170,-6.175170,1.6818,51,2.1783,57,1.2062,52,0.4604,44,98
99,Es,1.331047,5.944735,0.776738,6.223660,-3.662282,-0.669986,1.7129,51,2.3794,57,1.2522,52,0.4480,44,99
100,Fm,0.887312,5.782202,0.472582,6.209572,-3.664524,-0.453189,1.7145,51,2.4614,52,1.3218,55,0.4367,44,100


In [6]:
# Make sure we have all of the AFLOW columns:

print(aflow_df.columns)

Index(['compound', 'auid', 'aurl', 'spacegroup_relax', 'Pearson_symbol_relax',
       'Egap', 'catalog', 'nspecies'],
      dtype='object')


In [7]:
# Now we need to add some features here according to Wolverton 2016 paper:
# https://www.nature.com/articles/npjcompumats201628#Sec14

#A general-purpose machine learning framework for predicting properties of
# inorganic materials

# Atomic Number, Mendeleev Number,
# Atomic Weight, Melting Temperature, Periodic Table Column, Row,
# Covalent Radius, Electronegativity*, # s Valence Electrons, p Valence Electrons, d Valence Electrons,
# f Valence Electrons, Total # Valance Electrons, Unfilled s States†, Unfilled p States†, Unfilled d States†,
# Unfilled f States†, Total # Unfilled States†, Specific Volume of 0 K Ground State‡,
# Magnetic Moment (per atom) of 0 K ground state‡, Space Group Number of 0 K Ground State‡

# This shouldn't be too computationally heavy since we only have 102 rows. Let's create a function
# that grabs some mendeleev data:

def get_mendeleev_data(data_name, atomic_number_vec):
    """Return data as a column
    
    Args:
        data_name: Name of the data we want to get from mendeleev library.
        atomic_number_vec: List containing the atomic numbers for which we want the elemental
            data from mendeleev.
    Returns:
        List containing the elemental data.
    """
    elemental_data_list = np.zeros(len(atomic_number_vec))
    for i, atomic_number in enumerate(atomic_number_vec):
        mend_atom = mendeleev.element(int(atomic_number))
        elemental_data_list[i] = getattr(mend_atom, data_name)
    return elemental_data_list

# Let's get the covalent_radius_bragg as a test:

desired_properties_list = [
    'atomic_weight',
    'mendeleev_number',  # describes the atomic environment, gets larger in the right corner of periodic table.
    'melting_point',
    'period',  # Row of periodic table
#     'group',  # Column of the periodic table
    'covalent_radius_cordero',
    'en_allen',  # electronegativity Allen
    'en_ghosh',  # electronegativty Ghosh
    'en_pauling',
    'atomic_volume',
#     'period'  # Period in the periodic table.
]

for feature in desired_properties_list:
    print(feature)
    atomic_data_df[feature] = get_mendeleev_data(feature, atomic_data_df['atomic_number'])

atomic_data_df['n_valence'] = [mendeleev.element(x).nvalence() for x in atomic_data_df['atomic_number']]

atomic_data_df['HOMO_LUMO_diff'] = atomic_data_df['HOMO'] - atomic_data_df['LUMO']

atomic_weight
mendeleev_number
melting_point
period
covalent_radius_cordero
en_allen
en_ghosh
en_pauling
atomic_volume


In [10]:
enlarged_atomic_features_data_csv_path = (
    home_dir + '/elem_rf/enlarged_elemental_data.csv')
atomic_data_df.to_csv(enlarged_atomic_features_data_csv_path)

In [9]:
atomic_data_df.columns

Index(['Element', 'EA_half', 'IP_half', 'EA_delta', 'IP_delta', 'HOMO', 'LUMO',
       'rs', 's index', 'rp', 'p index', 'rd', 'd index', 'rf', 'f index',
       'atomic_number', 'atomic_weight', 'mendeleev_number', 'melting_point',
       'covalent_radius_cordero', 'en_allen', 'en_ghosh', 'en_pauling',
       'atomic_volume', 'n_valence', 'HOMO_LUMO_diff'],
      dtype='object')

In [1]:
import pandas as pd
home_dir = '/home/dts/Documents/hu/jraph_MPEU/other_models'
enlarged_atomic_features_data_csv_path = (
    home_dir + '/elem_rf/enlarged_elemental_data.csv')
atomic_data_df = pd.read_csv(enlarged_atomic_features_data_csv_path, index_col=None)
aflow_csv_data_source = '/band_gap_data/classification_data.csv'
# Read the data into a dataframe:
aflow_df = pd.read_csv(home_dir+aflow_csv_data_source, index_col=0)

In [2]:
# Ok now we want to create a new dataframe with the atomic features added in.
# Import our data prep module.
import data_prep as dp

data_prep_obj = dp.DataPrep(
    ase_db_path='/None',
    functional='pbe',
    elemental_features_csv=enlarged_atomic_features_data_csv_path)

features_list = [
       'EA_half']

# 'IP_half', 'EA_delta', 'IP_delta', 'HOMO', 'LUMO',
#        'rs', 's index', 'rp', 'p index', 'rd', 'd index', 'rf', 'f index',
#        'atomic_number', 'atomic_weight', 'mendeleev_number', 'melting_point',
#        'covalent_radius_cordero', 'en_allen', 'en_ghosh',
#        'en_pauling', 'atomic_volume', 'n_valence', 'HOMO_LUMO_diff']

compound_name_list = aflow_df['compound'][0:2000]

feature_engineered_data_df_with_compound = data_prep_obj.get_features_df(
    compound_name_list=compound_name_list, features_list=features_list)


In [ ]:
classification_data_x_and_y_features_data_csv_path = (
    home_dir+'/elem_rf/enlarged_elemental_data.csv')
feature_engineered_data_df_with_compound.to_csv(classification_data_x_and_y_features_data_csv_path)

In [ ]:
feature_engineered_data_df = feature_engineered_data_df_with_compound.drop(columns=['compound_name'])

print(feature_engineered_data_df.columns)

y_data = aflow_df['Egap']

In [ ]:
feature_engineered_data_df.describe()

In [ ]:
feature_engineered_data_df.columns

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    feature_engineered_data_df, y_data, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


regr = RandomForestRegressor(max_depth=50, random_state=42)
regr.fit(X_train, y_train)

In [ ]:
y_pred = regr.predict(X_test)

In [ ]:
# Now get statistics on the fit:

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print(f'The mean absolute error of the fit is: {mean_absolute_error(y_test, y_pred)}')
print(f'The mean root squared error of the fit is: {np.sqrt(mean_squared_error(y_test, y_pred))}')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_test, y_pred, '.')
plt.xlim([0.0, 2.5])
plt.ylim([0.0, 2.5])
plt.xlabel('True Band Gap (eV)')
plt.ylabel('Predicted Band Gap (eV)')
x = np.arange(0, 2.5, 0.1)
plt.plot(x, x, '-')

# What happens if we predict the mean

In [ ]:
# What happens if we simply predict the mean of the data? What MAE do we get?
y_mean = np.mean(y_train)
y_mean_vec = np.ones(len(y_test))*y_mean
print(f'The mean absolute error of predicting the mean is: {mean_absolute_error(y_test, y_mean_vec)}')


# Linear Model

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)
print(f' the regression test R2 score is: {reg.score(X_test, y_test)}')
print(f'The reg.coef_ is: {reg.coef_}')
print('The reg.intercept_ is {reg.intercept_}')

y_pred= reg.predict(X_test)

In [ ]:
print(f'The mean absolute error of the fit is: {mean_absolute_error(y_test, y_pred)}')
print(f'The mean root squared error of the fit is: {np.sqrt(mean_squared_error(y_test, y_pred))}')
import matplotlib.pyplot as plt

plt.plot(y_test, y_pred, '.')
plt.xlim([0.0, 2.5])
plt.ylim([0.0, 2.5])
plt.xlabel('True Band Gap (eV)')
plt.ylabel('Predicted Band Gap (eV)')
x = np.arange(0, 2.5, 0.1)
plt.plot(x, x, '-')

# Correlation of Features

In [ ]:
import seaborn as sns
# feature_engineered_data_df = feature_engineered_data_df.reset_index()
# aflow_df = aflow_df.reset_index()

print(type(aflow_df.compound[0]))
print(feature_engineered_data_df_with_compound.columns)
feature_engineered_data_df_with_compound = feature_engineered_data_df_with_compound.rename(
    columns={"compound_name": "compound"})
combined_df = pd.merge(feature_engineered_data_df_with_compound, aflow_df, on='compound', how='outer')

print(combined_df.columns)
combined_df = combined_df.drop(
    columns=['aurl', 'ldau_type', 'auid', 'compound', 'spacegroup_relax', 'Pearson_symbol_relax',
             'catalog', 'dft_type', 'energy_atom', 'level_0', 'index'])
print(combined_df.columns)

print(combined_df)


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
plt.title("Correlation Plot")
sns.heatmap(combined_df)